# Exercise for Week 12: Association Analysis

---

This week work is optional and slightly differs from the previous assignments.

The exercise focuses on a single question. A few hints and functions are provided which should pave the way to use the _apriori_ algorithm to find the two most frequent item collections in 250,000 orders.

The goal is to find a set of 4 items that are bought together in more than 340 orders.

## Apriori algorithm
The apriori algorithm is commonly used for association analysis in scenarios like e-commerce. The objective is to find which items are commonly bought toghether.

Since big e-commerce sites usually have tens of thousands of different items and millions of different historical orders, an exhaustive brute-force search of which items are commonly bought together is unfeasible.

The **Apriori** algorithm allows to find all the combinations of _k_ items that appear at least in _t_ different orders. The computation is much faster than brute-force approaches because the algorithm simplifies the search by recursive elimination of item combinations that do not satisfy the search constraints.

The recursive elimination of item combinations is illustrated in the image below:

<img src="data/combination-graph.png">

The image represents a scenario with 5 different items _a,b,c,d,e_ and we need to find the combinations of such items up to length _k_ with a threshold _t_. In this example the combination _ab_ is not present in at least _t_ orders and it is therefore discarded. Since _ab_ does not satisfy the threshold, any other itemset containing _ab_ will also be discarded. The combinations are computed adding one extra element at a time (until _k_ elements) and the search space is iteratively reduced by just keeping these combinations that do not satisfy the threshold.

## Implementation example
We provide you some functions that will help implement an apriori algorithm. First, we will run the functions on a toy dataset to understand precisely what they do. Then, you will have to use the helper functions to implement a complete apriori loop.

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import itertools as it
pd.__version__

'0.24.2'

In [4]:
pd.options.display.max_rows = 10

⚠️ Pandas version at least `0.25.0` is required for this assignment to run efficiently.

### Toy dataset

We will start with an example dataset. Here we have a list of 6 orders, each containing a variable number of products (from p0 to p6). The presence/absence of a product in the order is indicated by 1 and 0, respectively.

| order_id | p0 | p1 | p2 | p3 | p4 | p5 |
|----------|----|----|----|----|----|----|
| 0        | 1  | 1  | 1  | 0  | 0  | 0  |
| 1        | 1  | 1  | 0  | 0  | 0  | 0  |
| 2        | 0  | 0  | 0  | 1  | 1  | 0  |
| 3        | 1  | 1  | 1  | 1  | 0  | 0  |
| 4        | 1  | 1  | 1  | 0  | 1  | 0  |
| 5        | 0  | 0  | 0  | 0  | 1  | 1  |

This order information can be represented by a matrix containing 0 and 1. However, our real dataset will have millions of rows and thousands of columns, making it unfeasible to operate on the full matrix. Order matrices are usually very sparse, i.e. they contain relatively many more 0's than 1's, because each order usually contains just a few items out of all the available. We can exploit this property to store our matrix in sparse format, where the position of 1's is represented by a row containing the order and the item id. 

The same matrix could be represented in sparse format as follows:

| order_id | product_id |
|----------|------------|
| 0        | p0         |
| 0        | p1         |
| 0        | p2         |
| 1        | p0         |
| 1        | p1         |
| 2        | p3         |
| ...      | ...        |


In [5]:
toydata = pd.read_csv('data/toydata.csv')
toydata

,order_id,product_id
0,0,0
1,0,1
2,0,2
3,1,0
4,1,1
...,...,...
12,4,1
13,4,2
14,4,4
15,5,4


### Objective
The objective of this example is to find combinations of _k=3_ products that appear in at least _t=3_ orders.

In [3]:
t = 3

### Computing combination frequencies
The strategy is to compute the combination frequencies for all combinations of length _k=1_ and remove these products that do not appear in any of the combinations that satisfy the threshold _t_. Once certain these products have been removed, we can safely proceed to delete the orders that do not have at least _3_ items! Then we increase _k_ and repeat the same process.

#### Product combinations
The function below has two arguments:
- `df`: a DataSet representing a sparse matrix with two columns _order_id_ and _product_id_. 
- `k`: the length of the combinations

and it returns a tuple with two Series:
- `combinations_per_order`: the product combinations of length _k_ present in each order.
- `orders_per_combination`: the amount of orders that contain each combination of lengh _k_.

In [6]:
def product_combinations(df, k):
    # Group elements by order_id and compute element combinations of lenght k
    gr = df.groupby('order_id')['product_id'].apply(lambda x: list(it.combinations(sorted(x), k)))

    # Stack combinations for the same order
    # pd.Series().explode() function is only available in pandas version > 0.25
    v = [int(x) for x in pd.__version__.split('.')]
    if v[0] < 1 and v[1] < 25:
        # use inefficient explode
        co = gr.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'product_id'})
    else: 
        co = pd.DataFrame(gr.explode())
        co.reset_index(level=0, inplace=True)

    # Retrieve orders for each combination
    oc = co.groupby('product_id')['order_id'].count()
    
    # Return
    combinations_per_order = co
    orders_per_combination = oc
    return combinations_per_order, orders_per_combination

Run the cell below to compute the `order_combinations` of length `k=2` of `toydata`:

In [7]:
combinations_per_order, orders_per_combination = product_combinations(toydata,2)

The `combinations_per_order` show all the combinations of products with their associated _order_id_ , one combination per row:

In [8]:
combinations_per_order.head(10)

,order_id,product_id
0,0,"(0, 1)"
1,0,"(0, 2)"
2,0,"(1, 2)"
0,1,"(0, 1)"
0,2,"(3, 4)"
0,3,"(0, 1)"
1,3,"(0, 2)"
2,3,"(0, 3)"
3,3,"(1, 2)"
4,3,"(1, 3)"


Conversely, `orders_per_combination` counts the number of orders in which each combination appears:

In [9]:
orders_per_combination.head()

product_id
(0, 1)    4
(0, 2)    3
(0, 3)    1
(0, 4)    1
(1, 2)    3
Name: order_id, dtype: int64

### Selecting infrequent combinations
Now it's time to reduce the search space for the next combinations. Note that `orders_per_combination` comes in specially handy to target these combinations that do not satisfy the threshold _t_.

Let's count the frequency of each combination of two items:

In [10]:
orders_per_combination

product_id
(0, 1)    4
(0, 2)    3
(0, 3)    1
(0, 4)    1
(1, 2)    3
         ..
(1, 4)    1
(2, 3)    1
(2, 4)    1
(3, 4)    1
(4, 5)    1
Name: order_id, Length: 11, dtype: int64

It is now clear that only `(0,1)`, `(0,2)` and `(1,2)` appear `t=3` or more times in the dataset and therefore they are the only potential subsets of our target (remember that we are looking for combinations of 3 items that appear in at least **3** orders!).

The rest do not stand a chance to form combinations 3 items present 3 or more times, hence we will proceed to remove these combinations from our candidate list.

In [17]:
viable_combinations = orders_per_combination[orders_per_combination >= t].index
viable_combinations

Index([(0, 1), (0, 2), (1, 2)], dtype='object', name='product_id')

### Removing infrequent products

Our search space has now reduced to three potential item combinations: `(0,1)`, `(0,2)` and `(1,2)`. This has an important implication: only products `0`, `1` and `2` are actually relevant! The rest are not subsets of any viable combination and can be therefore removed from our dataset.

We provide the following helper function to retrieve all the unique products present in a list of product combinations:

In [18]:
def get_products_from_combinations(combinations):
    return np.unique(np.array([list(x) for x in combinations]).ravel())

Run the cell below to extract all the viable products from `viable_combinations`:

In [19]:
viable_products = get_products_from_combinations(viable_combinations)
viable_products

array([0, 1, 2])

Now we can proceed to eliminate the infrequent products from our dataset:

In [20]:
toydata = toydata[toydata.product_id.isin(viable_products)]
toydata

,order_id,product_id
0,0,0
1,0,1
2,0,2
3,1,0
4,1,1
7,3,0
8,3,1
9,3,2
11,4,0
12,4,1


Note how our dataset has reduced from 17 to 11 entries.

### Removing small orders

Now that we have deleted several products, some orders may have been left with less than _k_ items. If an order contains less than _k_ items it does not stand a chance to have a frequent combination of length _k_!

This step is simple, we just need to compute the number of products in each order and remove these orders whose size is less than _k_:

In [21]:
def order_sizes(df):
    return df.groupby('order_id').apply(len)

Run the cell below to compute the size of the remaining orders:

In [22]:
viable_order_sizes = order_sizes(toydata)
viable_order_sizes

order_id
0    3
1    2
3    3
4    3
dtype: int64

Order 1 has only 2 elements, so we can further reduce our search space by removing it!

In [23]:
toydata = toydata[toydata.order_id.isin(viable_order_sizes[viable_order_sizes >= 3].index)]
toydata

,order_id,product_id
0,0,0
1,0,1
2,0,2
7,3,0
8,3,1
9,3,2
11,4,0
12,4,1
13,4,2


And finally our search space has reduced to only 9 entries.

### Next iteration

At this point we have already reduced our search space based on the information obtained for _k=2_. We would now repeat the same process with _k=3,4..._ until the desired combination length.

In this simple example we are interested in finding combinations of _k=3_ items. We can quickly find the combinations on the reduced search space by repeating the same steps:

In [24]:
combinations_per_order, orders_per_combination = product_combinations(toydata,3)

Just by checking the frequency of our remaining combinations we find out that the item set `(0,1,2)` appears three times on our dataset and is the only combination that satisfies our conditions:

In [25]:
orders_per_combination

product_id
(0, 1, 2)    3
Name: order_id, dtype: int64

## Insta-Cart dataset

### The data

The dataset for this exercise is a subset of data taken from the ["Insta-Cart"](https://www.kaggle.com/c/instacart-market-basket-analysis/data) dataset on Kaggle. The dataset is in sparse matrix format:

In [26]:
data = pd.read_csv('data/insta_cart_subset.csv').sort_values(by=['order_id'])
print("Matrix entries: {}".format(data.shape[0]))
print("Number of unique products: {}".format(len(data.product_id.unique())))
print("Number of orders: {}".format(len(data.order_id.unique())))
data.head()

Matrix entries: 2686226
Number of unique products: 41670
Number of orders: 250000


,order_id,product_id
0,7,34050
1,7,46802
14,13,25952
12,13,36086
11,13,23020


### Objective

Your objective is to find the 4 items that were bought together more than 340 times.

### Algorithm
Implement the apriori loop making use of the helper functions defined above. Take into account the following considerations:
- Use the helper functions rather than trying to implement the loop from scratch.
- This is a long computation. It may take up to 30 minutes, depending on the efficiency of your code.
- You can use an itemset size _k=1_ at the first iteration or manually remove infrequent orders and products and start the loop at _k=2_.
- Computation may take long, we recommend you to check the first loop iteration line by line to avoid computation errors at the last iterations.
- It is good practice to copy the dataset `data` before the iteration. This will ensure that each time the loop is ran it starts with the original dataset. You can rename it to `orders` like:
```python
orders = data.copy()
# Your loop below (use orders instead of data)
```

### ⚠️⚠️⚠️ Grading considerations
This algorithm is way too complex to be computed in Vocareum autograder routines. If you submit this assignment with your apriori loop uncommented, **grading will likely fail due to timeout**. The answer must be hardcoded (copied literally) in the grading cell instead.

Follow these steps:
- Implement the algorithm below, you can use as many code cells as you need. To create a code cell select a cell (single click) and type 'b'. This will create a code cell below.
- Run your algorithm in your notebook before submitting.
- Once you get the output from your algorithm, copy the _product_id_ of each item in the itemset into a list and assign it to `ans1`.
- **Comment all your code** before submitting. Then submit your assignment.

If the **Python kernel crashes** in vocareum during computation:
- This likely means you are using too much memory.
- You can download this notebook (`File > Download as > Notebook`) and the dataset (<a href="data/insta_cart_subset.csv">click here to download the dataset</a>, then copy it to a folder `data` in your computer). Continue the computation locally in your computer, then copy the answer in the answer cell. You will need a computer with at least 4GB of RAM.

### Your implementation of apriori algorithm

### Answer cell

In [ ]:
### GRADED
# Assign here the product_ids of the items that were bought together in more than 340 orders.

# Your answer shoud look like:
# ans1 = [21341,12302,1002,4296]

ans1 = []

###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


### Lookup function for product names  

If you are interested, a single product ID or a list of product IDs may be looked up using the function below 

In [ ]:
prod_filepath = "data/products.csv"
products = pd.read_csv(prod_filepath)
def product_lookup(product_ids):
    try:
        len(product_ids)
        names = [products[products.product_id == pid].iloc[0,1] for pid in product_ids]
    except:
        names = products[products.product_id == product_ids].iloc[0,1]
    
    return names

In [ ]:
product_lookup(13176)